<h1 align="center">
  <a href="https://uptrain.ai">
    <img width="300" src="https://user-images.githubusercontent.com/108270398/214240695-4f958b76-c993-4ddd-8de6-8668f4d0da84.png" alt="uptrain">
  </a>
</h1>

<h1 style="text-align: center;">Fine-tuning a Large-Language Model</h1>

### Install Required packages
- [PyTorch](https://pytorch.org/get-started/locally/): Deep learning framework.
- Hugging Face Transformers(https://huggingface.co/docs/transformers/installation): To use pretrained state-of-the-art models.
- [Hugging Face Datasets](https://pypi.org/project/datasets/): Use public Hugging Face datasets
- [IPywidgets](https://ipywidgets.readthedocs.io/en/stable/user_install.html): For interactive notebook widgets

In [7]:
!pip install torch transformers datasets ipywidgets uptrain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.3/48.3 kB 837.9 kB/s eta 0:00:0031m1.5 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 5.8 MB/s eta 0:00:00m eta 0:00:010:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.2/15.2 MB 5.8 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 5.7 MB/s eta 0:00:00m eta 0:00:010:01:01


  Using cached threadpoolctl-3.1.0-py3-none-any.whl (14 kB)
  Using cached joblib-1.2.0-py3-none-any.whl (297 kB)


In [5]:
!wget https://github.com/uptrain-ai/uptrain/raw/main/examples/4_bert_llm/model_constants.py

--2023-02-06 19:19:08--  https://github.com/uptrain-ai/uptrain/raw/main/examples/4_bert_llm/model_constants.py
Resolving github.com (github.com)... 20.207.73.82
Connecting to github.com (github.com)|20.207.73.82|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/uptrain-ai/uptrain/main/examples/4_bert_llm/model_constants.py [following]
--2023-02-06 19:19:09--  https://raw.githubusercontent.com/uptrain-ai/uptrain/main/examples/4_bert_llm/model_constants.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 185 [text/plain]
Saving to: ‘model_constants.py’

model_constants.py  100%[===================>]     185  --.-KB/s    in 0s      

2023-02-06 19:19:09 (4.52 MB/s) - ‘model_constants.py’ saved [185/1

In [6]:
!wget https://github.com/uptrain-ai/uptrain/raw/main/examples/4_bert_llm/model_train.py

--2023-02-06 19:19:11--  https://github.com/uptrain-ai/uptrain/raw/main/examples/4_bert_llm/model_train.py
Resolving github.com (github.com)... 20.207.73.82
Connecting to github.com (github.com)|20.207.73.82|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/uptrain-ai/uptrain/main/examples/4_bert_llm/model_train.py [following]
--2023-02-06 19:19:12--  https://raw.githubusercontent.com/uptrain-ai/uptrain/main/examples/4_bert_llm/model_train.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1819 (1.8K) [text/plain]
Saving to: ‘model_train.py’

model_train.py      100%[===================>]   1.78K  --.-KB/s    in 0s      

2023-02-06 19:19:12 (13.3 MB/s) - ‘model_train.py’ saved [1819/1819]



In [10]:
!wget https://github.com/uptrain-ai/uptrain/raw/main/examples/4_bert_llm/helper_funcs.py

--2023-02-06 19:20:40--  https://github.com/uptrain-ai/uptrain/raw/main/examples/4_bert_llm/helper_funcs.py
Resolving github.com (github.com)... 20.207.73.82
Connecting to github.com (github.com)|20.207.73.82|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/uptrain-ai/uptrain/main/examples/4_bert_llm/helper_funcs.py [following]
--2023-02-06 19:20:41--  https://raw.githubusercontent.com/uptrain-ai/uptrain/main/examples/4_bert_llm/helper_funcs.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4332 (4.2K) [text/plain]
Saving to: ‘helper_funcs.py’

helper_funcs.py     100%[===================>]   4.23K  --.-KB/s    in 0s      

2023-02-06 19:20:42 (9.59 MB/s) - ‘helper_funcs.py’ saved [4332/4332]



In [1]:
from transformers import AutoModelForMaskedLM, AutoTokenizer
from model_constants import *
from model_train import retrain_model
from helper_funcs import *
import json
import uptrain
import torch
# this ensures that the current MacOS version is at least 12.3+
print(torch.backends.mps.is_available())
# this ensures that the current current PyTorch installation was built with MPS activated.
print(torch.backends.mps.is_built())
dtype = torch.float
device = torch.device("mps")

True
True


In [2]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForMaskedLM.from_pretrained(model_checkpoint)
testing_text = "Nike shoes are very [MASK]."
original_model_outputs = test_model(model, testing_text)

In [36]:
def nike_text_present_func(inputs, outputs, gts=None, extra_args={}):
    is_present = []
    for input in inputs["text"]:
        this_present = "nike" in input.lower()
        is_present.append(bool(this_present))
    return is_present

uptrain_save_fold_name = "uptrain_smart_data_bert"
nike_text_present = uptrain.Signal("Nike Text Present", nike_text_present_func)

cfg = {
    'checks': [{
        'type': uptrain.Anomaly.EDGE_CASE,
        "signal_formulae": nike_text_present
    }],

    # Define where to save the retraining dataset
    'retraining_folder': uptrain_save_fold_name,
    
    # Define when to retrain, define a large number because we
    # are not retraining yet
    'retrain_after': 10000000000
}

framework = uptrain.Framework(cfg)

Deleting the folder:  uptrain_smart_data_bert


In [37]:
raw_dataset = create_sample_dataset("data.json")
with open(raw_dataset) as f:
    all_data = json.load(f)

for sample in all_data['data']:
    inputs = {'data': {'text': [sample['text']]}}
    framework.log(inputs = inputs, outputs = None)

retraining_dataset = create_dataset_from_csv(uptrain_save_fold_name + "/1/smart_data.csv", "text", "retrain_dataset.json")

50  edge cases identified out of  197  total samples
100  edge cases identified out of  397  total samples
150  edge cases identified out of  597  total samples
200  edge cases identified out of  797  total samples
250  edge cases identified out of  997  total samples
300  edge cases identified out of  1197  total samples
350  edge cases identified out of  1397  total samples
400  edge cases identified out of  1597  total samples
450  edge cases identified out of  1797  total samples
500  edge cases identified out of  1997  total samples
550  edge cases identified out of  2197  total samples
600  edge cases identified out of  2397  total samples
650  edge cases identified out of  2597  total samples
700  edge cases identified out of  2797  total samples
750  edge cases identified out of  2997  total samples
800  edge cases identified out of  3197  total samples
850  edge cases identified out of  3397  total samples
900  edge cases identified out of  3597  total samples
950  edge cases 

In [25]:
retrain_model(model, retraining_dataset)
retrained_model_outputs = test_model(model, testing_text)

Using custom data configuration default-d2d70f9c8f339009
Found cached dataset json (/Users/linuz/.cache/huggingface/datasets/json/default-d2d70f9c8f339009/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51)


  0%|          | 0/1 [00:00<?, ?it/s]

Loading cached processed dataset at /Users/linuz/.cache/huggingface/datasets/json/default-d2d70f9c8f339009/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51/cache-917a4b96a4c3b86e.arrow
Loading cached processed dataset at /Users/linuz/.cache/huggingface/datasets/json/default-d2d70f9c8f339009/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51/cache-a1682c8182e25c1e.arrow
Loading cached split indices for dataset at /Users/linuz/.cache/huggingface/datasets/json/default-d2d70f9c8f339009/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51/cache-431ca2af5112a606.arrow and /Users/linuz/.cache/huggingface/datasets/json/default-d2d70f9c8f339009/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51/cache-edd67712f81abd40.arrow
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to 

The following columns in the training set don't have a corresponding argument in `DistilBertForMaskedLM.forward` and have been ignored: word_ids. If word_ids are not expected by `DistilBertForMaskedLM.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 93
  Num Epochs = 3
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 6


>>>Before training, Perplexity: 2.32


Epoch,Training Loss,Validation Loss
1,1.086400,1.024335
2,1.009500,0.996117
3,0.697200,0.758949


The following columns in the evaluation set don't have a corresponding argument in `DistilBertForMaskedLM.forward` and have been ignored: word_ids. If word_ids are not expected by `DistilBertForMaskedLM.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 11
  Batch size = 64
The following columns in the evaluation set don't have a corresponding argument in `DistilBertForMaskedLM.forward` and have been ignored: word_ids. If word_ids are not expected by `DistilBertForMaskedLM.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 11
  Batch size = 64
The following columns in the evaluation set don't have a corresponding argument in `DistilBertForMaskedLM.forward` and have been ignored: word_ids. If word_ids are not expected by `DistilBertForMaskedLM.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 11
  Batch size = 64


Training completed. Do not forget to shar

>>>After training, Perplexity: 2.55


In [35]:


import pandas as pd
# Create Nike review training dataset

# Download the dataset from the url, zip it and copy the csv file here
raw_nike_reviews_dataset = create_dataset_from_csv("/Users/linuz/Code/Machine_Learning/flipr/kaggle dataset/web_scraped.csv", "Content", "raw_nike_reviews_data.json")

In [40]:
test_model(model, 'Nike shoes and [MASK]')

['shoes', 'socks', 'jeans', 'sneakers', 'boots']